# Preprocessing datasets for analysis with scvi-tools

## Dependencies

```{note}
Running the following cell will install tutorial dependencies on Google Colab only. It will have no effect on environments other than Google Colab.
```

In [1]:
!pip install --quiet scvi-colab
from scvi_colab import install

install()

INFO     scvi-colab: Installing scvi-tools.                                                                        
INFO     scvi-colab: Install successful. Testing import.                                                           


/usr/local/lib/python3.11/dist-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/usr/local/lib/python3.11/dist-packages/docrep/decorators.py:43: SyntaxWarning: 'param_continuous_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


## Imports and preparing files

In [2]:
import gzip
import os
import tempfile
from pathlib import Path

import anndata as ad
import mudata as md
import muon
import numpy as np
import pooch
import scanpy as sc
import scvi
import seaborn as sns
import torch

In [3]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

INFO: Seed set to 0
INFO:lightning.fabric.utilities.seed:Seed set to 0


Last run with scvi-tools version: 1.3.0


```{note}
You can modify `save_dir` below to change where the data files for this tutorial are saved.

You can modify `file_name` below to the name of the dataset you would like to preprocess. This file will end with .h5ad or .h5 depending on which model you plan to use.
```

In [4]:
sc.set_figure_params(figsize=(6, 6), frameon=False)
sns.set_theme()
torch.set_float32_matmul_precision("high")
save_dir = tempfile.TemporaryDirectory()

%config InlineBackend.print_figure_kwargs={"facecolor": "w"}
%config InlineBackend.figure_format="retina"

## scRNA-seq

Here we demonstrate preprocessing of an scRNA-seq dataset using cells from the lung atlas integration task from the [scIB manuscript](https://www.biorxiv.org/content/10.1101/2020.05.22.111161v2).

In [5]:
adata_path = os.path.join(save_dir.name, "lung_atlas.h5ad")

adata = sc.read(
    adata_path,
    backup_url="https://figshare.com/ndownloader/files/24539942",
)
adata

  0%|          | 0.00/972M [00:00<?, ?B/s]

AnnData object with n_obs × n_vars = 32472 × 15148
    obs: 'dataset', 'location', 'nGene', 'nUMI', 'patientGroup', 'percent.mito', 'protocol', 'sanger_type', 'size_factors', 'sampling_method', 'batch', 'cell_type', 'donor'
    layers: 'counts'

This dataset already has counts stored in layers, and `adata.X` contains log transformed scran normalized expression. If this is not the case for your dataset, you can preserve the raw counts, then normalize and log transform them with
```python
adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
```

Below we perform gene selection while keeping the full dimension normalized data in `adata.raw`. We obtain variable genes from each dataset and take their intersections.

In [6]:
adata.raw = adata  # keep full dimension safe
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=2000,
    flavor="seurat_v3",
    layer="counts",
    subset=True,
    batch_key="batch",  # Change depending on the batch key for your dataset
)

/usr/local/lib/python3.11/dist-packages/scanpy/preprocessing/_highly_variable_genes.py:74: UserWarning: `flavor='seurat_v3'` expects raw count data, but non-integers were found.
  warnings.warn(


```{important}
We see a warning about the data not containing counts. This is due to some of the samples in this dataset containing SoupX-corrected counts. scvi-tools models will run for non-negative real-valued data, but we strongly suggest checking that these possibly non-count values are intended to represent pseudocounts, and not some other normalized data, in which the variance/covariance structure of the data has changed dramatically.
```

## scATAC-seq

To demonstrate preprocessing of scATAC-seq data, we use a 5k PBMC sample dataset from 10X. We use the pooch package to download the data.

In [7]:
def download_data(save_path: str, fname: str = "atac_pbmc_5k") -> str:
    """Download the data files."""
    data_paths = pooch.retrieve(
        url="https://cf.10xgenomics.com/samples/cell-atac/1.2.0/atac_pbmc_5k_nextgem/atac_pbmc_5k_nextgem_filtered_peak_bc_matrix.tar.gz",
        known_hash="78e536a1508108fa5bd3b411a7484809c011f3403800369b20db05bdbfeb2284",
        fname=fname,
        path=save_path,
        processor=pooch.Untar(),
        progressbar=True,
    )
    return str(Path(data_paths[0]).parent)

In [8]:
data_path = download_data(save_dir.name)

100%|████████████████████████████████████████| 114M/114M [00:00<00:00, 107GB/s]
Untarring contents of '/tmp/tmpne5dtz2o/atac_pbmc_5k' to '/tmp/tmpne5dtz2o/atac_pbmc_5k.untar'


In [9]:
adata = scvi.data.read_10x_atac(data_path)
adata

AnnData object with n_obs × n_vars = 4585 × 115554
    obs: 'batch_id'
    var: 'chr', 'start', 'end'

We use Scanpy here to filter out peaks that are rarely detected, so that the model trains faster:

In [10]:
print("# regions before filtering:", adata.shape[-1])

# compute the threshold: 5% of the cells
min_cells = int(adata.shape[0] * 0.05)
# in-place filtering of regions
sc.pp.filter_genes(adata, min_cells=min_cells)

print("# regions after filtering:", adata.shape[-1])

# regions before filtering: 115554
# regions after filtering: 33142


## CITE-seq

Here we demonstrate preprocessing of CITE-seq data on integrated PBMC10k and PBMC5k datasets available from 10X Genomics. We will subset to the 17 proteins shared between the datasets.

In [11]:
def download_data(save_path: str, fname: str = "CITE-seq_pbmc_10k") -> str:
    """Download the data files."""
    if fname == "CITE-seq_pbmc_10k":
        hash = "md5:26d53ffe08b5f7d3b28df61b592d51fb"
        url = "https://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_10k_protein_v3/pbmc_10k_protein_v3_filtered_feature_bc_matrix.tar.gz"
    else:
        hash = "md5:9be3d672b0445b944ca06a507c3d780b"
        url = "https://cf.10xgenomics.com/samples/cell-exp/3.1.0/5k_pbmc_protein_v3/5k_pbmc_protein_v3_filtered_feature_bc_matrix.tar.gz"

    data_paths = pooch.retrieve(
        url=url,
        known_hash=hash,
        fname=fname,
        path=save_path,
        processor=pooch.Untar(),
        progressbar=True,
    )
    return str(Path(data_paths[0]).parent)

In [12]:
data_path1 = download_data(save_dir.name)
data_path2 = download_data(save_dir.name, "CITE-seq_pbmc_5k")

100%|█████████████████████████████████████| 48.7M/48.7M [00:00<00:00, 22.9GB/s]
Untarring contents of '/tmp/tmpne5dtz2o/CITE-seq_pbmc_10k' to '/tmp/tmpne5dtz2o/CITE-seq_pbmc_10k.untar'
100%|█████████████████████████████████████| 36.2M/36.2M [00:00<00:00, 19.3GB/s]
Untarring contents of '/tmp/tmpne5dtz2o/CITE-seq_pbmc_5k' to '/tmp/tmpne5dtz2o/CITE-seq_pbmc_5k.untar'


In [13]:
mdata1 = muon.read_10x_mtx(data_path1)
mdata2 = muon.read_10x_mtx(data_path2)

/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalit

In [14]:
mdata1

MuData object with n_obs × n_vars = 7865 × 33555
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	7865 x 33538
      var:	'gene_ids', 'feature_types'
    prot:	7865 x 17
      var:	'gene_ids', 'feature_types'

In [15]:
mdata2

MuData object with n_obs × n_vars = 5247 × 33570
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	5247 x 33538
      var:	'gene_ids', 'feature_types'
    prot:	5247 x 32
      var:	'gene_ids', 'feature_types'

We make var names unique, store raw counts in layers, and normalize and log transform counts. Then we perform gene selection.

https://github.com/YosefLab/totalVI_reproducibility/blob/master/data/data_filtering_scripts/pbmc_10k/pbmc_10k.py

In [16]:
# TODO: Should I do more filtering as in the above link? Or is it too specific for this?
# min genes, mitochondrial reads, n_genes, n_counts, protein library size,
# genes that encode proteins, etc.

# The processing done in the below link seems much more minimal so not sure which is better here

https://docs.scvi-tools.org/en/stable/tutorials/notebooks/multimodal/totalVI.html#using-own-data

In [17]:
def filter_pbmc(mdata):
    mdata.mod["rna"].var_names_make_unique()
    mdata.mod["rna"].layers["counts"] = mdata.mod["rna"].X.copy()
    sc.pp.normalize_total(mdata.mod["rna"])
    sc.pp.log1p(mdata.mod["rna"])

    sc.pp.highly_variable_genes(
        mdata.mod["rna"],
        n_top_genes=4000,
        flavor="seurat_v3",
        layer="counts",
    )
    # Place subsetted counts in a new modality
    mdata.mod["rna_subset"] = mdata.mod["rna"][:, mdata.mod["rna"].var["highly_variable"]].copy()
    mdata = md.MuData(mdata.mod)

    return mdata

In [18]:
mdata1 = filter_pbmc(mdata1)
mdata2 = filter_pbmc(mdata2)

/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:931: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_com

In [19]:
# TODO: Are the above warnings an issue?

In [20]:
# TODO: not sure if below code is best way to combine the datasets

# First filter to shared proteins and genes by doing inner join
rna_c = ad.concat([mdata1.mod["rna_subset"], mdata2.mod["rna_subset"]])
rna_c.obs_names_make_unique()

prot_c = ad.concat([mdata1.mod["prot"], mdata2.mod["prot"]])
prot_c.obs_names_make_unique()

# Todo: do I need to still include the original "rna" modality for any reason?
mdata = md.MuData({"rna_subset": rna_c, "prot": prot_c})

/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by d

In [21]:
# we need to work with dense and not sparse matrices:
mdata["prot"].X = mdata["prot"].X.toarray()
mdata["rna_subset"].X = mdata["rna_subset"].X.toarray()
mdata.mod["rna_subset"].layers["counts"] = mdata.mod["rna_subset"].layers["counts"].toarray()

In [22]:
mdata.update()

/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


In [23]:
mdata

MuData object with n_obs × n_vars = 13112 × 2193
  2 modalities
    rna_subset:	13112 x 2176
      layers:	'counts'
    prot:	13112 x 17

## Multiome

```{important}
MultiVI requires the datasets to use shared features. scATAC-seq datasets need to be processed to use a shared set of peaks.
```

Here we use a 10k PBMCs dataset from 10X to demonstrate preprocessing for multiome datasets.

In [24]:
def download_data(save_path: str, fname: str = "pbmc_10k"):
    data_paths = pooch.retrieve(
        url="https://cf.10xgenomics.com/samples/cell-arc/2.0.0/pbmc_unsorted_10k/pbmc_unsorted_10k_filtered_feature_bc_matrix.tar.gz",
        known_hash="872b0dba467d972aa498812a857677ca7cf69050d4f9762b2cd4753b2be694a1",
        fname=fname,
        path=save_path,
        processor=pooch.Untar(),
        progressbar=True,
    )
    data_paths.sort()

    for path in data_paths:
        with gzip.open(path, "rb") as f_in:
            with open(path.replace(".gz", ""), "wb") as f_out:
                f_out.write(f_in.read())

    return str(Path(data_paths[0]).parent)

In [25]:
data_path = download_data(save_dir.name)

100%|████████████████████████████████████████| 375M/375M [00:00<00:00, 355GB/s]
Untarring contents of '/tmp/tmpne5dtz2o/pbmc_10k' to '/tmp/tmpne5dtz2o/pbmc_10k.untar'


We read the dataset as a mudata, where we have two modalities: 'rna' and 'atac'. For multiome datasets, using mudata is preferred over anndata.

In [26]:
mdata = muon.read_10x_mtx(data_path)
mdata

/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/usr/local/lib/python3.11/dist-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 12012 × 148458
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	12012 x 36601
      var:	'gene_ids', 'feature_types'
    atac:	12012 x 111857
      var:	'gene_ids', 'feature_types'

In [27]:
mdata.mod["rna"].var_names_make_unique()
mdata.mod["atac"].var_names_make_unique()

Next we zero out the values of each modality for 1/3 of the observations, to create a mixed single-modal and multimodal dataset. We do this so that we can later demonstrate the ability to work with mixed data.

In [28]:
# still working on this cell

# n = mdata.n_obs // 3
# mdata.mod["rna"][n:, :] = 0
# mdata.mod["atac"][: 2 * n, :] = 0

In [29]:
mdata

MuData object with n_obs × n_vars = 12012 × 148458
  var:	'gene_ids', 'feature_types'
  2 modalities
    rna:	12012 x 36601
      var:	'gene_ids', 'feature_types'
    atac:	12012 x 111857
      var:	'gene_ids', 'feature_types'

In [30]:
print(mdata.mod["rna"].shape)
sc.pp.filter_genes(mdata.mod["rna"], min_cells=int(mdata.mod["rna"].shape[0] * 0.01))
print(mdata.mod["rna"].shape)

(12012, 36601)
(12012, 13634)


In [31]:
print(mdata.mod["atac"].shape)
sc.pp.filter_genes(mdata.mod["atac"], min_cells=int(mdata.mod["atac"].shape[0] * 0.01))
print(mdata.mod["atac"].shape)

(12012, 111857)
(12012, 83899)


## Spatial transciptomics

To demonstrate preprocessing for spatial transcriptomics, we use data from a comparative study of murine lymph nodes, comparing wild-type with a stimulation after injection of a mycobacteria. We have at disposal a 10x Visium dataset as well as a matching scRNA-seq dataset from the same tissue.

In [32]:
url1 = "https://github.com/romain-lopez/DestVI-reproducibility/blob/master/lymph_node/deconvolution/ST-LN-compressed.h5ad?raw=true"
url2 = "https://github.com/romain-lopez/DestVI-reproducibility/blob/master/lymph_node/deconvolution/scRNA-LN-compressed.h5ad?raw=true"
out1 = "data/ST-LN-compressed.h5ad"
out2 = "data/scRNA-LN-compressed.h5ad"

First, let’s load the single-cell data. We profiled immune cells from murine lymph nodes with 10x Chromium, as a control / case study to study the immune response to exposure to a mycobacteria (refer to DestVI paper for more info). It contains the raw counts (DestVI always takes raw counts as input).

In [33]:
sc_adata = sc.read(out2, backup_url=url2)

  0%|          | 0.00/76.0M [00:00<?, ?B/s]

In [34]:
# let us filter some genes
G = 2000
sc.pp.filter_genes(sc_adata, min_counts=10)

sc_adata.layers["counts"] = sc_adata.X.copy()

sc.pp.highly_variable_genes(
    sc_adata, n_top_genes=G, subset=True, layer="counts", flavor="seurat_v3"
)

sc.pp.normalize_total(sc_adata, target_sum=10e4)
sc.pp.log1p(sc_adata)
sc_adata.raw = sc_adata

Load the spatial data

In [35]:
st_adata = sc.read(out1, backup_url=url1)

  0%|          | 0.00/12.8M [00:00<?, ?B/s]

In [36]:
st_adata.layers["counts"] = st_adata.X.copy()
st_adata.obsm["spatial"] = st_adata.obsm["location"]

sc.pp.normalize_total(st_adata, target_sum=10e4)
sc.pp.log1p(st_adata)
st_adata.raw = st_adata

Here we must ensure that the two datasets have a common gene subset.

In [37]:
# filter genes to be the same on the spatial data
intersect = np.intersect1d(sc_adata.var_names, st_adata.var_names)
st_adata = st_adata[:, intersect].copy()
sc_adata = sc_adata[:, intersect].copy()
G = len(intersect)